### Procédure pour créer le fichier .csv contenant les entités nommées ainsi que leurs fonctions grammaticales

In [1]:
import pandas as pd  
import spacy
import en_core_web_sm 

In [5]:
nlp = en_core_web_sm.load() # On charge une fonction de nlp anglaise de spacy

In [2]:
data_raw = pd.read_csv('pictures_sample.csv') # On charge la base de données dans un DataFrame

c:\users\dimit\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (3,21,22,33,35,39,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_english = data_raw[data_raw.Language == 'EN'] # On ne conserve que les images dont la langue renseignée est l'anglais

# On constate que certaines descriptions sont identiques à l'auteur près, si on affiche les deux premières descriptions on a:

print(data_english.Description[0])
print(data_english.Description[1])

# On va donc créer une fonction qui supprime le nom de l'auteur (si il y en a un) dans chaque description

def author_del(text):
    text_s = text
    while(text_s[-1] != '\n' and len(text_s) > 1): # On veut supprimer la dernière ligne sautée, sans pour autant supprimer
                                                   # tout le texte
        text_s = text_s[:-1]
    if(len(text_s) <= 1):
        return(text)
    return(text_s[:-1])

data_english.Description = data_english.Description.apply(author_del)          # On applique la fonction précédente au DF
data_english.drop_duplicates(subset='Description', keep='first', inplace=True) # On supprime les doublons
ind = list(data_english.index)

# On a divisé par deux le nombre d'image

print(data_english.shape)

c:\users\dimit\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
c:\users\dimit\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


(82608, 54)

In [6]:
# On crée une liste dans laquelle nous allons stocker l'application de la fonction nlp à toutes nos descriptions

nlp_list = []

for desc in data_english.Description:
    nlp_list.append(nlp(desc))

In [7]:
# On définit une fonction pour retrouver une entité nommée dans une liste de mots ordonnée

# i correspond au nombre de mots de l'entité nommée, et j au mot auquel on s'intéresse dans la description

def ent_in_words(i, j, desc):
    if(j+i-1 >= len(desc)):
        return("")
    words = ""
    for k in range(i):
        words += (desc[j+k].text + " ")
    return(words[:-1]) # On renvoit les i prochains mots à partir du j-ième mot

# Comme la fonction précédente sauf que l'on renvoit les fonctions grammaticales

def dep_in_ent(i, j, desc):
    l = []
    for k in range(i):
        l.append(desc[j+k].dep_)
    return(l)

# On supprime les listes vides contenues dans l

def clear_blanks(l):
    L = []
    for i in range(len(l)):
        if(len(l[i]) != 0):
            L.append(l[i])
    return(L)

In [9]:
# On crée une liste qui servira à stocker les entités nommées ainsi que leurs fonctions grammaticales pour chaque description

final_list = []

# On la remplie

for k in range(len(nlp_list)): 
    desc = nlp_list[k]
    
    # Pour chaque description, on crée une liste de liste de tuple:
    # - chaque liste dans la liste correspondra aux différentes occurences de chaque entités nommées
    # - chaque tuple correspondra au nom de l'entité nommée ainsi que sa(ses) fonctions grammaticales correspond à 
    # cette occurence
    
    ents_desc = [[(ent.text, dep_in_ent(len(ent.text.split()), i, desc)) for i in range(len(desc))
                                                              if (ent.text == ent_in_words(len(ent.text.split()), i, desc)) 
                                                             and (ent.label_ == "PERSON" 
                                                               or ent.label_ == "NORP" 
                                                               or ent.label_ == "ORG"
                                                               or ent.label_ == "GPE" 
                                                               or ent.label_ == "EVENT"
                                                               or ent.label_ == "LOC")] for ent in nlp_list[k].ents]
    
    # On supprime les listes vides correspondant aux entités nommées dites inintéressantes
    
    ents_desc = clear_blanks(ents_desc)
    final_list.append(ents_desc)

In [10]:
# On sauvegarde la liste crée sous forme de DataFrame ayant un format particulier (car ses colonnes seront des objets, en
# particulier des listes)

ent_df = pd.DataFrame(final_list, dtype = 'O')

# Pour sauvegarder ce DataFrame il faut donc utilisé la méthode to_pickle et non pas to_csv

ent_df.to_pickle("ent.csv")

### Procédure pour lire le fichier .csv contenant les entités nommées ainsi que leurs fonctions grammaticales

In [11]:
# On lit le fichier avec read_pickle étant donné qu'il contient des objets (en particulier des listes)

ent_load = pd.read_pickle("ent.csv")

In [13]:
# On transforme le DataFrame en matrice afin de retrouver son format original

M = ent_load.to_numpy()
M[M == None] = 0 # On remplace les colonnes "None" par 0 (on ne peut pas les remplacer par des listes vides)

# On les remplacent donc après par des listes vides

for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        if M[i,j] == 0:
            M[i,j] = []

# On remplace chaque élément par lui-même dans une liste afin de pouvoir concaténer les colonnes dans une grosse liste via la
# méthode sum, mais pour ne pas avoir de listes vides on ne remplacent pas les listes vides par [[]].

for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        if M[i,j] != []:
            M[i,j] = [M[i,j]]

In [14]:
# Ainsi en sommant les colonnes, on obtient l'objet sous sa forme originale

M = list(M.sum(axis = 1))

In [17]:
# On importe le fichier des scores que l'on transforme en dictionnaire

scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}
for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]

In [18]:
# On crée une fonction qui à une liste de fonctions grammaticales renvoit le maximum des scores associés d'après le .csv

def dep_to_val(l):
    L = []
    for i in range(len(l)):
        if l[i] in dict_val.keys():
            L.append(dict_val[l[i]])
        else:
            # Si la fonction grammaticale n'est pas dans le fichier, on associe un score de 0
            L.append(0)
    return(max(L))

# On définit une fonction qui retournera l'ensembles des entités et leurs scores associés grâce à la fonction prédécente pour
# une description

def final(l):
    L = [] # Liste finale
    
    # Pour entité de la description
    
    for i in l:
        ll = [] # On crée sa liste finale associée
        
        # On l'a rempli par son nom et sa valeur associée
        
        for ii in i:
            ll.append((ii[0], dep_to_val(ii[1])))
        
        # On rajoute cette liste dans la liste finale
        
        L.append(ll)
        
    # Pour chaque entité de la liste finale, on somme les scores de l'entité
    
    for i in range(len(L)):
        name = L[i][0][0]
        score = 0
        for ii in L[i]:
            score += ii[1]
        L[i] = (name, score)
    return(L)

# On crée une fonction qui ne gardera qu'une seule fois plusieurs entités (leur score prennant déjà en compte le nombre
# d'apparition)

def clean_final(l):
    L = []
    for i in l:
        ll = []
        for ii in i:
            if ii not in ll:
                ll.append(ii)
        L.append(ll)
    return(L)

In [19]:
# On applique la fonction final à chaque description de notre liste

M = [final(m) for m in M]

In [20]:
# On nettoie les entités en double 

M = clean_final(M)

In [22]:
# On met tout ça sous forme de dictionnaire pour en faciliter l'utilisation

M = [dict(m) for m in M]

In [26]:
# Affichage des dictionnaires des dix premières descriptions

print(M[0:10])

[{'US': 0.7659574469999999, 'Sienna Miller': 1.0, 'Deauville US Film Festival': 0.872340426, 'Deauville': 0.8936170219999999, 'France': 0.5106382979999999}, {'Shatin': 0.021276596000000002, 'Hong Kong': 1.744680852}, {'AFP': 1.0, 'Christof Stache': 0.872340426}, {'Palestinian': 0.531914894, 'Jordan Valley': 0.936170213, 'Jiftlik': 0.872340426, 'Israeli': 1.2765957449999998, 'West Bank': 1.297872341, 'Benjamin Netanyahu': 0.425531915, 'Netanyahu': 1.425531915, 'Palestinians': 0.872340426, 'Arab': 0.531914894, 'the United Nations': 0.46808510600000003, 'the European Union': 0.46808510600000003}, {'Turkish': 0.531914894, 'Ruhsar Pekcan': 1.0, 'US': 0.765957447, 'Wilbur Ross': 0.106382979, 'Ankara': 0.872340426}, {'Mini Electric': 0.021276596000000002, 'Mini': 0.8936170219999999, 'the IAA Car Show': 0.872340426, 'Frankfurt': 1.297872341, 'biennial International Auto Show': 1.0}, {'BMW': 1.638297873, 'German': 0.531914894, 'Frankfurt': 1.297872341, 'Germany': 0.5106382979999999, 'the Intern